# Problem Set #4
- MACS 40200, Dr. Evans 
- Name: Kento Yoshizawa (CNET: kyoshizawa) 
- Date: February 17, 2020

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as intgr
import scipy.optimize as opt
import scipy.stats as sts
#from scipy import stats, special
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [17]:
macro = np.loadtxt('data/NewMacroSeries.txt', delimiter=',', dtype = 'float128')
macro.shape

(100, 5)

## (a)

In [3]:
def BM_simulation(unif_vals, params, k1):
    T, S = unif_vals.shape
    
    alpha, rho, mu, sig = params
    beta = 0.99
    
    eps = sts.norm.ppf(unif_vals, loc = 0, scale = sig)
    
    zt = np.zeros_like(unif_vals)
    for s in range(S):
        zvec = np.zeros(T + 1)
        zvec[0] = mu
        for t in range(T):
            z = rho * zvec[t] + (1 - rho) * mu + eps[t,s]
            zvec[t+1] = z
        zt[:,s] = zvec[1:]
    
    kt = np.zeros((T + 1, S))
    for s in range(S):
        kvec = np.zeros(T + 1)
        kvec[0] = k1
        for t in range(T):
            k = alpha * beta * np.exp(zt[t,s]) * kvec[t]**alpha
            kvec[t + 1] = k
        kt[:,s] = kvec
    
    wt = (1 - alpha) * np.exp(zt) * kt[:-1] ** alpha
    
    rt = alpha * np.exp(zt) * kt[:-1] ** (alpha - 1)
    
    ct = wt + rt * kt[:-1] - kt[1:]
    
    yt = np.exp(zt) * kt[:-1] ** alpha
    
    return np.array([ct, kt[:-1], wt, rt, yt])

In [4]:
def data_moments(data):
    if data.ndim == 2:
        ct, kt, yt = data[:,0], data[:,1], data[:,4]
        mom1 = ct.mean()
        mom2 = kt.mean()
        mom3 = (ct/yt).mean()
        mom4 = yt.var()
        mom5 = np.corrcoef(ct[:-1], ct[1:])[0,1]
        mom6 = np.corrcoef(ct, kt)[0,1]
        
    elif data.ndim == 3:
        ct, kt, yt = data[0], data[1], data[4]
        mom1 = ct.mean(axis = 0)
        mom2 = kt.mean(axis = 0)
        mom3 = (ct/yt).mean(axis = 0)
        mom4 = yt.var(axis = 0)
        mom5 = np.zeros(ct.shape[1])
        mom6 = np.zeros(ct.shape[1])
        for s in range(ct.shape[1]):
            mom5[s] = np.corrcoef(ct[:-1,s], ct[1:,s])[0,1]
            mom6[s] = np.corrcoef(ct[:,s], kt[:,s])[0,1]
      
    return np.array([mom1, mom2, mom3, mom4, mom5, mom6])

In [5]:
def err_vec(data, unif_vals, params, simple):
    k1 = data[:,1].mean()
    sim_data = BM_simulation(unif_vals, params, k1)
    sim_moms  = np.array([sm.mean() for sm in data_moments(sim_data)])
    data_moms = data_moments(data)
    
    if simple:
        err_vec = sim_moms - data_moms
    else:
        err_vec = (sim_moms - data_moms)/data_moms
    
    return err_vec

In [6]:
def criterion(params, *args):
    data, unif_vals, W_hat = args
    
    err = err_vec(data, unif_vals, params, simple=False)
    crit_val = err.T @ W_hat @ err 
    
    return crit_val

In [19]:
## Define random drawing
T = 100
S = 1000
unif_vals = sts.uniform.rvs(0, 1, size = (T, S))

In [20]:
## Initial guess
alpha_init = 0.5 
rho_init   = 1.0
mu_init    = 10.0
sigma_init = 0.5
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])

## Weighting matrix
W_hat = np.eye(6)
## SMM arguments
smm_args1 = (macro, unif_vals, W_hat)

## bounds
bounds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))


%time results1 = opt.minimize(criterion, params_init, args = (smm_args1), \
                              method = 'L-BFGS-B', bounds = bounds, tol = 1e-14)
results1

CPU times: user 3min 4s, sys: 905 ms, total: 3min 5s
Wall time: 3min 6s


      fun: 4.1587535017625606066e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.83461662e-06, -3.64053806e-06, -2.66385671e-07, -8.00770473e-06])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 475
      nit: 54
   status: 0
  success: True
        x: array([0.42100674, 0.92092418, 9.92846415, 0.0887103 ])

In [21]:
print('Estimated values using SMM with the Identity matrix\n \
    - alpha: {0:.4f} \n \
    - rho  : {1:.4f} \n \
    - mu   : {2:.4f} \n \
    - sigma: {3:.4f}'.format(*results1.x))
print('\nThe vector of moment difference at the optimum:\n', err_vec(macro, unif_vals, results1.x, True))
print('\nThe value of the minimized criterion function: {}'.format(results1.fun))

Estimated values using SMM with the Identity matrix
     - alpha: 0.4210 
     - rho  : 0.9209 
     - mu   : 9.9285 
     - sigma: 0.0887

The vector of moment difference at the optimum:
 [ 6.64378394e+03 -4.79954634e+03 -9.96676021e-04 -8.72471552e+06
  3.09624593e-04 -3.05684656e-04]

The value of the minimized criterion function: 4.1587535017625606e-06


In [22]:
def Jac_err6(data, unif_vals, params, simple=False):
    
    h = 1e-4
    K = len(params)
    R = 6
    
    Jac_err = np.zeros((R, K))
    I = np.eye(K)
    
    for k in range(K):
        e = I[k]
        ub = params + h * e
        lb = params - h * e
        
        Jac_err[:, k] = ((err_vec(data, unif_vals, ub, simple) - 
                          err_vec(data, unif_vals, lb, simple)) / (2 * h)).flatten()
        
    return Jac_err

In [23]:
d_err6_1 = Jac_err6(macro, unif_vals, results1.x)
SigHat1 = (1/S) * np.linalg.inv(d_err6_1.T @ W_hat @ d_err6_1)
std1 = np.sqrt(np.diag(SigHat1))
print('Std.errors for the estimated parameters\n \
    - alpha: {0:.4f}\n \
    - rho  : {1:.4f}\n \
    - mu   : {2:.4f}\n \
    - sigma: {3:.4f}'.format(*std1))

Std.errors for the estimated parameters
     - alpha: 0.0095
     - rho  : 0.0474
     - mu   : 0.1604
     - sigma: 0.0208


## (b)

In [24]:
k1 = macro[:,1].mean()
sim_data_opt = BM_simulation(unif_vals, results1.x, k1)

In [25]:
def get_Err_mat(data, unif_vals, params, simple):
    R = 6
    S = unif_vals.shape[1]
    
    k1 = data[:,1].mean()
    sim_data  = BM_simulation(unif_vals, params, k1)
    sim_moms  = data_moments(sim_data)
    data_moms = data_moments(data)
    
    Err_mat = np.zeros((R, S))
    
    for r in range(R):
        if simple:
            Err_mat[r] = sim_moms[r] - data_moms[r]
        else:
            Err_mat[r] = (sim_moms[r] - data_moms[r])/data_moms[r]
    
    return Err_mat

In [26]:
Err_mat = get_Err_mat(macro, unif_vals, results1.x, False)
VCV = (1 / S) * (Err_mat @ Err_mat.T)
W_hat2 = np.linalg.inv(VCV)

In [27]:
## SMM arguments
smm_args2 = (macro, unif_vals, W_hat2)

%time results2 = opt.minimize(criterion, results1.x, args=(smm_args2), \
                                method='L-BFGS-B', bounds= bounds)
results2

CPU times: user 1min 22s, sys: 518 ms, total: 1min 23s
Wall time: 1min 23s


      fun: 0.7036447731927209488
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([7.52745739e-05, 9.29317929e-06, 5.52392875e-06, 1.53045003e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 205
      nit: 34
   status: 0
  success: True
        x: array([0.42070787, 0.92255142, 9.93085939, 0.08758171])

In [29]:
print('Estimated values using SMM with the Identity matrix\n \
    - alpha: {0:.4f} \n \
    - rho  : {1:.4f} \n \
    - mu   : {2:.4f} \n \
    - sigma: {3:.4f}'.format(*results2.x))
print('\nThe vector of moment differences at the optimum:\n', err_vec(macro, unif_vals, results2.x, True))
print('\nThe value of the minimized criterion function: {}'.format(results2.fun))

Estimated values using SMM with the Identity matrix
     - alpha: 0.4207 
     - rho  : 0.9226 
     - mu   : 9.9309 
     - sigma: 0.0876

The vector of moment differences at the optimum:
 [-3.29128836e+04 -4.07536113e+04 -7.00793714e-04 -5.89233346e+11
  1.02625470e-03  3.97361220e-04]

The value of the minimized criterion function: 0.7036447731927209


In [30]:
d_err6_2 = Jac_err6(macro, unif_vals, results2.x)
SigHat2 = (1/S) * np.linalg.inv(d_err6_2.T @ W_hat2 @ d_err6_2)
std2 = np.sqrt(np.diag(SigHat2))
print('Std.errors for the estimated parameters\n \
    - alpha: {0:.5f}\n \
    - rho  : {1:.5f}\n \
    - mu   : {2:.5f}\n \
    - sigma: {3:.5f}'.format(*std2))

Std.errors for the estimated parameters
     - alpha: 0.00002
     - rho  : 0.00198
     - mu   : 0.00297
     - sigma: 0.00070


---

In [37]:
import pandas as pd

results = np.array([results1.x, std1, results2.x, std2])

pd.DataFrame(data = results,
             index = ['Estimators with Identity matrix',
                      '(Std.Err)',
                      'Estimators with Two-step matrix', 
                      '(Std.Err)'],
             columns = [r'$\alpha$',r'$\rho$',r'$\mu$',r'$\sigma$'])

,$\alpha$,$\rho$,$\mu$,$\sigma$
Estimators with Identity matrix,0.421007,0.920924,9.928464,0.088710
(Std.Err),0.009499,0.047431,0.160386,0.020755
Estimators with Two-step matrix,0.420708,0.922551,9.930859,0.087582
(Std.Err),0.000017,0.001978,0.002971,0.000698
